# E-Commerce Customer Review Prediction Model : Shubham Agrawal

Importing the Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind

Loading Each File

In [ ]:
customers = pd.read_csv('olist_customers_dataset.csv')
geolocation = pd.read_csv('olist_geolocation_dataset.csv')
order_items = pd.read_csv('olist_order_items_dataset.csv')
order_payments = pd.read_csv('olist_order_payments_dataset.csv')
order_reviews = pd.read_csv('olist_order_reviews_dataset.csv')
orders = pd.read_csv('olist_orders_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')
sellers = pd.read_csv('olist_sellers_dataset.csv')
product_category_translation = pd.read_csv('product_category_name_translation.csv')

# Cleaning the Datasets

*1. Customers
Dataset*

In [ ]:
# Step 1: Overview of the dataset
print("Dataset Overview:")
print(customers.info())

# Step 2: Remove duplicate rows
customers = customers.drop_duplicates()

# Step 3: Standardize categorical data
customers['customer_state'] = customers['customer_state'].str.upper()  # Convert state codes to uppercase
customers['customer_city'] = customers['customer_city'].str.lower()    # Convert city names to lowercase
string_columns = customers.select_dtypes(include=['object']).columns
customers[string_columns] = customers[string_columns].apply(lambda x: x.str.strip())  # Strip whitespace

# Step 4: Validate and drop invalid geographic states
valid_states = ['AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MT', 'MS', 'MG', 'PA', 'PB', 'PR',
                'PE', 'PI', 'RJ', 'RN', 'RS', 'RO', 'RR', 'SC', 'SP', 'SE', 'TO']
customers = customers[customers['customer_state'].isin(valid_states)]

# Step 5: Check for Missing Values
print("\nChecking for missing values...")
missing_values = customers.isnull().sum()
print(missing_values)
#There are no missing values


*2. Geolocation Dataset*

In [ ]:
# Step 1: Overview of the dataset
print("Dataset Overview:")
print(geolocation.info())

# Step 2: Remove duplicates
geolocation = geolocation.drop_duplicates()

# Step 3: Standardize categorical columns
geolocation['geolocation_state'] = geolocation['geolocation_state'].str.upper()
geolocation['geolocation_city'] = geolocation['geolocation_city'].str.lower()
string_columns = geolocation.select_dtypes(include=['object']).columns
geolocation[string_columns] = geolocation[string_columns].apply(lambda x: x.str.strip())

# Step 4: Handle missing values
missing_values = geolocation.isnull().sum()
print(missing_values)


# Step 5: Validate and remove invalid latitude/longitude values
geolocation = geolocation[
    (geolocation['geolocation_lat'].between(-90, 90)) &
    (geolocation['geolocation_lng'].between(-180, 180))
]



*3. Order items Dataset*

In [ ]:
# Step 1: Overview of the dataset
print("Dataset Overview:")
print(order_items.info())

# Step 2: Remove duplicate rows
order_items = order_items.drop_duplicates()

# Step 3: Handle invalid or missing values
# Step 3.1 Convert 'shipping_limit_date' to datetime format
if 'shipping_limit_date' in order_items.columns:
    order_items['shipping_limit_date'] = pd.to_datetime(order_items['shipping_limit_date'])

# Step 3.2: Check for Missing Values
print("\nChecking for missing values...")
missing_values = order_items.isnull().sum()
print(missing_values)

# Step 3.4 Ensure 'price' and 'freight_value' are valid
order_items = order_items[(order_items['price'] > 0) & (order_items['freight_value'] >= 0)]

# Step 4: Standardize string columns
string_columns = order_items.select_dtypes(include=['object']).columns
order_items[string_columns] = order_items[string_columns].apply(lambda x: x.str.strip())




*4. Order Payments Dataset*

In [ ]:
# Step 1: Overview of the dataset
print("Dataset Overview:")
print(order_payments.info())

# Step 2: Remove duplicate rows
order_payments = order_payments.drop_duplicates()

# Step 3: Handle invalid or missing values
missing_values = order_payments.isnull().sum()
print(missing_values)

# Ensure 'payment_value' is positive or at least 0
order_payments = order_payments[order_payments['payment_value'] >= 0]

# Ensure 'payment_installments' is greater than 0
order_payments = order_payments[order_payments['payment_installments'] > 0]

# Remove rows where 'payment_type' is 'not_defined'
order_payments = order_payments[order_payments['payment_type'] != 'not_defined']

# Step 4: Standardize string columns
# Convert 'payment_type' to lowercase
order_payments['payment_type'] = order_payments['payment_type'].str.lower()

# Strip whitespace from string columns
string_columns = order_payments.select_dtypes(include=['object']).columns
order_payments[string_columns] = order_payments[string_columns].apply(lambda x: x.str.strip())

# Step 5: Check for continuity of 'payment_sequential'
def is_sequential(seq):
    # Check if the sequence starts from 1 and is continuous
    return sorted(seq) == list(range(1, len(seq) + 1))

# Identify valid order IDs where 'payment_sequential' is continuous
valid_order_ids = (
    order_payments.groupby('order_id')['payment_sequential']
    .apply(is_sequential)
    .reset_index()
    .rename(columns={'payment_sequential': 'is_sequential'})
)

# Merge back with the original dataset to filter only valid rows
order_payments = order_payments.merge(valid_order_ids, on='order_id')
order_payments = order_payments[order_payments['is_sequential']]
order_payments = order_payments.drop(columns=['is_sequential'])

# Step 6: Aggregate data to ensure one row per 'order_id'
aggregated_payments = order_payments.groupby('order_id').agg({
    'payment_sequential': 'max',  # Highest payment sequential
    'payment_type': lambda x: ', '.join(sorted(x.unique())),  # Unique payment types, comma-separated
    'payment_installments': 'max',  # Highest installments
    'payment_value': 'sum'  # Total payment value
}).reset_index()

# Display the aggregated DataFrame
print(aggregated_payments.head())

# Save the aggregated DataFrame
aggregated_payments.to_csv('aggregated_payments.csv', index=False)
print("Aggregated data saved to 'aggregated_payments.csv'")




*5. Order Reviews Dataset*

In [ ]:
# Step 1: Overview of the dataset
print("Dataset Overview:")
print(order_reviews.info())

# Step 2.1: Check Missing Values
missing_values = order_reviews.isnull().sum()
print("\nMissing Values Summary:")
print(missing_values)

# Step 2.2: Check Missing Values Percentage
missing_percentage = (order_reviews.isnull().mean() * 100).round(2)  # Calculate percentage of missing values
print(missing_percentage)

# Step 3: Remove unnecessary columns with too high % of missing values
order_reviews = order_reviews.drop(columns=['review_comment_title', 'review_comment_message'], errors='ignore')

# Count duplicates in 'order_id'
duplicate_count = order_reviews.duplicated(subset='order_id').sum()
print(f"\nNumber of duplicate 'order_id' entries: {duplicate_count}")

# Step 4: Keep only the final row of reviews for each order
order_reviews = order_reviews.sort_values(by='review_answer_timestamp').drop_duplicates(subset='order_id', keep='last')

# Step 5: Handle missing values
# Drop rows with missing values in critical columns
critical_columns = ['review_id', 'order_id', 'review_score']
order_reviews = order_reviews.dropna(subset=critical_columns)

# Step 6: Convert date fields to datetime
date_columns = ['review_creation_date', 'review_answer_timestamp']
for col in date_columns:
    order_reviews[col] = pd.to_datetime(order_reviews[col], dayfirst=True)

# Step 7: Standardize string columns
string_columns = order_reviews.select_dtypes(include=['object']).columns
order_reviews[string_columns] = order_reviews[string_columns].apply(lambda x: x.str.strip())




*6. Orders Dataset*

In [ ]:
# Step 1: Overview of the dataset
print("Dataset Overview:")
print(orders.info())

# Step 2: Remove duplicate rows
orders = orders.drop_duplicates()

# Step 3: Missing values
missing_values = orders.isnull().sum()
print("\nMissing Values Summary:")
print(missing_values)

# Step 4: Handle missing values
# Drop rows with missing values in critical columns
critical_columns = ['order_id', 'customer_id', 'order_purchase_timestamp', 'order_delivered_customer_date', 'order_estimated_delivery_date']
orders = orders.dropna(subset=critical_columns)

# Step 6: Convert date fields to datetime
date_columns = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]
for col in date_columns:
    orders[col] = pd.to_datetime(orders[col])

# Step7: Validate and clean date relationships
# Ensure carrier delivery is after purchase,customer delivery is after carrier delivery
orders = orders[
   (orders['order_delivered_carrier_date'] >= orders['order_purchase_timestamp'])|
   (orders['order_approved_at'] >= orders['order_purchase_timestamp'])|
   (orders['order_delivered_customer_date'] >= orders['order_delivered_carrier_date'])
]




*7. Products Dataset*

In [ ]:
# Step 1: Overview of the dataset
print("Dataset Overview:")
print(products.info())

# Step 2: Remove duplicate rows
products = products.drop_duplicates()

# Step 3: Standardize and clean categorical columns
if 'product_category_name' in products.columns:
    products['product_category_name'] = products['product_category_name'].str.strip().str.lower()

# Step 4: Handle missing values
missing_values = products.isnull().sum()
print("\nMissing Values Summary:")
print(missing_values)

# Drop rows with missing values in critical columns
critical_columns = ['product_id', 'product_category_name']
products = products.dropna(subset=critical_columns)

# Step 5: Validate and clean numeric fields
# Ensure product weight, length, height, and width are positive
numeric_columns = [
    'product_weight_g',
    'product_length_cm',
    'product_height_cm',
    'product_width_cm'
]
for col in numeric_columns:
    if col in products.columns:
        products = products[products[col] > 0]




*8. Sellers Dataset*

In [ ]:
# Step 1: Overview of the dataset
print("Dataset Overview:")
print(sellers.info())

# Step 2: Remove duplicate rows
sellers = sellers.drop_duplicates()

# Step 3: Handle missing values
missing_values = sellers.isnull().sum()
print("\nMissing Values Summary:")
print(missing_values)

# Step 4: Standardize categorical columns
# Convert state codes to uppercase and city names to lowercase
sellers['seller_city'] = sellers['seller_city'].str.strip().str.lower()
sellers['seller_state'] = sellers['seller_state'].str.strip().str.upper()




*9. Translation Dataset*

In [ ]:
# Step 1: Overview of the dataset
print("Dataset Overview:")
print(product_category_translation.info())

# Step 2: Remove duplicate rows
product_category_translation = product_category_translation.drop_duplicates()

# Step 3: Handle missing values
missing_values = product_category_translation.isnull().sum()
print("\nMissing Values Summary:")
print(missing_values)

# Step 4: Standardize categorical columns
# Strip whitespace and convert to lowercase for consistency
product_category_translation['product_category_name'] = product_category_translation['product_category_name'].str.strip().str.lower()
product_category_translation['product_category_name_english'] = product_category_translation['product_category_name_english'].str.strip().str.lower()


#Merging the Datasets

In [ ]:
# Step 1: Merge customers with orders
customers_orders = pd.merge(customers, orders, on='customer_id', how='inner')

# Step 2: Merge customers_orders with order_reviews
customers_orders_reviews = pd.merge(customers_orders, order_reviews, on='order_id', how='inner')

# Step 3: Merge customers_orders_reviews with aggregated_payments
customers_orders_reviews_payments = pd.merge(customers_orders_reviews, aggregated_payments, on='order_id', how='inner')

# Step 4: Merge customers_orders_reviews_payments with order_items
customers_orders_items = pd.merge(customers_orders_reviews_payments, order_items, on='order_id', how='inner')

# Step 5: Merge customers_orders_items with products
customers_orders_items_products = pd.merge(customers_orders_items, products, on='product_id', how='inner')

# Step 6: Merge customers_orders_items_products with sellers
customers_orders_items_products_sellers = pd.merge(customers_orders_items_products, sellers, on='seller_id', how='inner')

# Step 7: Merge customers_orders_items_products_sellers with translation
final_merged_data = pd.merge(
    customers_orders_items_products_sellers,
    product_category_translation,
    left_on='product_category_name',
    right_on='product_category_name',
    how='left'
)

# Final merged dataset
final_merged_data.to_csv('final_merged_data.csv', index=False)
print("Final merged dataset saved to 'final_merged_data.csv'")

#Cleaning the Merged Dataset

 Step 1: Checking the missing values



In [ ]:
file_path = 'final_merged_data.csv'
final_data = pd.read_csv(file_path)

# Calculate missing values and their percentage
missing_data = final_data.isnull().sum().reset_index()
missing_data.columns = ['Column', 'Missing_Values']
missing_data['Percentage_Missing'] = (missing_data['Missing_Values'] / len(final_data)) * 100

# Print missing values and percentages
print("Missing Values and Percentages per Column:")
print(missing_data)

Step 2: Fill in the Missing values for Category names in English

In [ ]:
# Identify rows with missing translations
missing_translations = final_data[final_data['product_category_name_english'].isnull()]
print(missing_translations[['product_category_name', 'product_category_name_english']])

In [ ]:
translation_dict = {
    'portateis_cozinha_e_preparadores_de_alimentos': 'portable kitchen and food preparation devices',
    'pc_gamer': 'gaming pc'
}

# Replace missing translations using the dictionary
final_data['product_category_name_english'] = final_data['product_category_name_english'].fillna(
    final_data['product_category_name'].map(translation_dict)
)

# Verify if all missing values are replaced
print(final_data['product_category_name_english'].isnull().sum())

Step 3: Identifying and Removing Inconsistent Date Entries from the Dataset

In [ ]:
# Identify invalid rows
invalid_rows_condition = (
    (final_data['review_creation_date'] < final_data['order_delivered_customer_date']) |
    (final_data['review_creation_date'] < final_data['order_purchase_timestamp'])
)

# Count invalid rows
invalid_rows_count = final_data[invalid_rows_condition].shape[0]
print(f"Number of invalid rows (review date earlier than purchase or delivery date): {invalid_rows_count}")

# Remove invalid rows
cleaned_data = final_data[~invalid_rows_condition]

# Save cleaned data
cleaned_data.to_csv('cleaned_merged_data.csv', index=False)

In [ ]:
# Number of rows in the cleaned merged dataset
cleaned_merged_data = pd.read_csv('cleaned_merged_data.csv')
num_rows = len(cleaned_merged_data)
print(f"The cleaned merged dataset has {num_rows} rows.")

Step 4: Removing Duplicates

In [ ]:
#Duplicates : Remove orders with multiple items
# Group by order ID and count the number of items in each order
order_counts = cleaned_merged_data.groupby('order_id')['order_item_id'].count()

# Identify orders with only one item
single_item_orders = order_counts[order_counts == 1].index

# Filter the DataFrame to keep only orders with a single item
cleaned_merged_data_single_item = cleaned_merged_data[cleaned_merged_data['order_id'].isin(single_item_orders)]

num_rows_clean = len(cleaned_merged_data_single_item)
print(f"The further cleaned merged dataset has {num_rows_clean} rows.")

# Save the cleaned dataset without duplicates (orders with multiple items)
cleaned_merged_data_single_item.to_csv('cleaned_merged_data_single_item.csv', index=False)
print("Cleaned merged dataset (single item orders) saved to 'cleaned_merged_data_single_item.csv'")

Step 5: Filtering out Delivered Orders

In [ ]:
#Order Status : Remove orders that are not Delivered

# Load the dataset into Orders
orderStatus = pd.read_csv('cleaned_merged_data_single_item.csv')

# Remove rows where 'order_status' is not 'delivered'
orderStatus = orderStatus[orderStatus['order_status'].str.strip().str.lower() == 'delivered']

# Save the filtered dataset
filtered_orders_file_path = 'orders_only_delivered_single_item.csv'
orderStatus.to_csv(filtered_orders_file_path, index=False)

# Print summary
print(f"Filtered dataset saved with {orderStatus.shape[0]} rows where 'order_status' is 'delivered'.")

Step 6: Add columns for new features

In [ ]:
#New column : Delivery Time = Delivery Customer Data - Purchase Timestamp
#New column : Delivery Delay = Delivery Customer Data - Estimated Delivery Date
#New Column : Review to Delivery Time = Review Creation Date - Delivered Date
#New column : Freight Price Ratio = Freight value / price

In [ ]:
# Load the dataset
data = pd.read_csv('orders_only_delivered_single_item.csv')

# Convert relevant columns to datetime for calculations
data['order_purchase_timestamp'] = pd.to_datetime(data['order_purchase_timestamp'])
data['order_delivered_customer_date'] = pd.to_datetime(data['order_delivered_customer_date'])
data['order_estimated_delivery_date'] = pd.to_datetime(data['order_estimated_delivery_date'])
data['review_creation_date'] = data['review_creation_date'].apply(lambda x: x + ' 00:00:00' if len(x) == 10 else x)
data['review_creation_date'] = pd.to_datetime(data['review_creation_date'], format='%Y-%m-%d %H:%M:%S')

# New column: Delivery Time = Delivered Customer Date - Purchase Timestamp
data['delivery_time'] = (data['order_delivered_customer_date'] - data['order_purchase_timestamp']).dt.days

# New column: Delivery Customer Date - Estimated Delivery Date
data['delivery_delay'] = (data['order_delivered_customer_date'] - data['order_estimated_delivery_date']).dt.days

# New column: Review Creation Date - Delivered Date
data['review_to_delivery_time'] = (data['review_creation_date'] - data['order_delivered_customer_date']).dt.days

# New column: Ratio of Freight Value to Price
data['freight_price_ratio'] = data['freight_value'] / data['price']

# Save the updated dataset
output_file_path = 'orders_with_new_columns.csv'
data.to_csv(output_file_path, index=False)

# Print summary
print(f"Dataset updated with new columns and saved to {output_file_path}.")

Step 7: Outlier Analysis

In [ ]:
data = pd.read_csv('orders_with_new_columns.csv')

# Select numerical features specific to the provided dataset
numerical_features = data.select_dtypes(include=['float64', 'int64']).columns

# Set up the figure size
plt.figure(figsize=(20, 15))

# Plot histograms for each numerical feature
for i, feature in enumerate(numerical_features):
    plt.subplot((len(numerical_features) + 2) // 3, 3, i + 1)  # Create grid of subplots
    sns.histplot(data[feature], kde=True, color='skyblue', bins=30)
    plt.title(f'Histogram of {feature}', fontsize=12)
    plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
print(data.info())

# Feature Selection

After cleaning the datasets, moving onto comparing different variables to review scores to understand their impact through visualisation and statistical analysis.

In [ ]:
# Select numerical features
numerical_features = data.select_dtypes(include=['float64', 'int64'])

# Calculate the correlation matrix
correlation_matrix = numerical_features.corr()

# Plot the correlation heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', cbar_kws={'label': 'Correlation Coefficient'})
plt.title('Correlation Matrix for Numerical Features', fontsize=16)
plt.show()


*1. Customer State*

In [ ]:
# Count plot for review scores by customer state
plt.figure(figsize=(15, 6))
sns.countplot(data=data, x='customer_state', hue='review_score', palette='Set2')
plt.title('Review Score Distribution Across Customer States', fontsize=16)
plt.xlabel('Customer State', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.legend(title='Review Score', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Calculate the average review score for each customer state
state_avg_reviews = data.groupby('customer_state')['review_score'].mean().reset_index()

# Calculate the median of average review scores by state
median_state_score = state_avg_reviews['review_score'].median()

# Identify states in each group
high_performing_states = state_avg_reviews[state_avg_reviews['review_score'] >= median_state_score]['customer_state']
low_performing_states = state_avg_reviews[state_avg_reviews['review_score'] < median_state_score]['customer_state']

# Filter data for the two groups
group_high = data[data['customer_state'].isin(high_performing_states)]['review_score']
group_low = data[data['customer_state'].isin(low_performing_states)]['review_score']

# Perform t-test
t_stat, p_value = ttest_ind(group_high, group_low, equal_var=False)

# Print results
print("T-Test: Impact of Customer State on Review Scores")
print(f"T-Statistic: {t_stat:.4f}")
print(f"P-Value: {p_value:.4f}")

# Interpretation
if p_value < 0.05:
    print("The difference in review scores between high-performing and low-performing states is statistically significant.")
else:
    print("The difference in review scores between high-performing and low-performing states is not statistically significant.")


*2. Payment Type*

In [ ]:
# Count plot for review scores by payment type
plt.figure(figsize=(12, 6))
sns.countplot(data=data, x='payment_type', hue='review_score', palette='Set2')
plt.title('Review Score Distribution by Payment Type', fontsize=16)
plt.xlabel('Payment Type', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.legend(title='Review Score')
plt.show()


In [ ]:
# Step 1: Filter Data into Two Groups
credit_card_reviews = data[data['payment_type'] == 'credit_card']['review_score']
other_payment_reviews = data[data['payment_type'] != 'credit_card']['review_score']

# Step 2: Perform T-Test
t_stat, p_value = ttest_ind(credit_card_reviews, other_payment_reviews, equal_var=False)

# Step 3: Print Results
print("T-Test: Credit Card vs All Other Payment Methods")
print(f"T-Statistic: {t_stat:.4f}")
print(f"P-Value: {p_value:.4f}")

# Interpretation
if p_value < 0.05:
    print("The difference in review scores between Credit Card and All Other Payment Methods is statistically significant.")
else:
    print("The difference in review scores between Credit Card and All Other Payment Methods is not statistically significant.")


*3. Payment Installments*

In [ ]:
# Scatter plot for review scores by payment installments
plt.figure(figsize=(12, 6))
sns.scatterplot(data=data, x='payment_installments', y='review_score', color='purple', alpha=0.6)
plt.title('Scatter Plot: Payment Installments vs Review Score', fontsize=16)
plt.xlabel('Payment Installments', fontsize=12)
plt.ylabel('Review Score', fontsize=12)
plt.show()


In [ ]:
# Step 1: Calculate the median of payment installments
median_installments = data['payment_installments'].median()

# Step 2: Divide the data into two groups
low_installments = data[data['payment_installments'] < median_installments]['review_score']
high_installments = data[data['payment_installments'] >= median_installments]['review_score']

# Step 3: Perform T-Test
t_stat, p_value = ttest_ind(low_installments, high_installments, equal_var=False)

# Step 4: Print Results
print("T-Test: Low vs High Payment Installments")
print(f"T-Statistic: {t_stat:.4f}")
print(f"P-Value: {p_value:.4f}")

# Interpretation
if p_value < 0.05:
    print("The difference in review scores between low and high payment installments is statistically significant.")
else:
    print("The difference in review scores between low and high payment installments is not statistically significant.")



*4. Price*

In [ ]:
# Box plots for review scores by Prices
plt.figure(figsize=(12, 6))
sns.boxplot(data=data, x=pd.cut(data['price'], bins=5), y='review_score', palette='Set2')
plt.title('Box Plot: Price (Binned) vs Review Score', fontsize=16)
plt.xlabel('Price (Binned)', fontsize=12)
plt.ylabel('Review Score', fontsize=12)
plt.show()


In [ ]:
# Step 1: Calculate Median Price
median_price = data['price'].median()

# Step 2: Divide Data into Low and High Price Groups
low_price_group = data[data['price'] < median_price]['review_score']
high_price_group = data[data['price'] >= median_price]['review_score']

# Step 3: Perform T-Test
t_stat, p_value = ttest_ind(low_price_group, high_price_group, equal_var=False)

# Step 4: Print Results
print("T-Test: Low vs High Price Groups")
print(f"T-Statistic: {t_stat:.4f}")
print(f"P-Value: {p_value:.4f}")

# Interpretation
if p_value < 0.05:
    print("The difference in review scores between Low and High price groups is statistically significant.")
else:
    print("The difference in review scores between Low and High price groups is not statistically significant.")


*5. Freight Value*

In [ ]:
# Create bins for freight value to group similar ranges
data['freight_value_bins'] = pd.cut(data['freight_value'], bins=10)

# Box plot of review scores across freight value ranges
plt.figure(figsize=(14, 8))
sns.boxplot(data=data, x='freight_value_bins', y='review_score', palette='Set2')
plt.title('Review Score Distribution Across Freight Value Ranges', fontsize=16)
plt.xlabel('Freight Value Range', fontsize=12)
plt.ylabel('Review Score', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.show()


In [ ]:
# Split data into two groups based on review scores
bad_reviews = data[data['review_score'] <= 3]['freight_value']
good_reviews = data[data['review_score'] > 3]['freight_value']

# Perform t-test
t_stat, p_value = ttest_ind(bad_reviews, good_reviews, equal_var=False)  # Use Welch's t-test (unequal variances)

# Display results
print("T-Test Results: Freight Value and Review Scores")
print(f"T-Statistic: {t_stat:.4f}")
print(f"P-Value: {p_value:.4f}")

if p_value < 0.05:
    print("The difference in freight value between the two groups is statistically significant.")
else:
    print("The difference in freight value between the two groups is not statistically significant.")


*6. Product Category*

In [ ]:
# Calculate mean review scores for each category
avg_review_scores = data.groupby('product_category_name_english')['review_score'].mean()

# Sort the average review scores
avg_review_scores_sorted = avg_review_scores.sort_values()

# Select the top 10 and lowest 10 categories
lowest_10_categories = avg_review_scores_sorted.head(10)
top_10_categories = avg_review_scores_sorted.tail(10)

# Combine the top and lowest categories into one DataFrame
top_and_lowest = pd.concat([lowest_10_categories, top_10_categories])

# Bar plot
plt.figure(figsize=(14, 8))
sns.barplot(
    x=top_and_lowest.index,
    y=top_and_lowest.values,
    palette='coolwarm'
)
plt.title('Top 10 and Lowest 10 Product Categories by Average Review Score', fontsize=16)
plt.xlabel('Product Category', fontsize=12)
plt.ylabel('Average Review Score', fontsize=12)
plt.xticks(rotation=90, ha='right')  # Rotate category names for readability
plt.show()


In [ ]:
# Comparing two random categories to test for significance

# Step 1: Define the two categories to compare
category_1 = 'sports_leisure'
category_2 = 'furniture_decor'

# Step 2: Filter the data for the two selected categories
category_1_scores = data[data['product_category_name_english'] == category_1]['review_score']
category_2_scores = data[data['product_category_name_english'] == category_2]['review_score']

# Step 3: Perform t-test
t_stat, p_value = ttest_ind(category_1_scores, category_2_scores, equal_var=False)  # Use Welch's t-test

# Step 4: Display results
print(f"T-Test Results for {category_1} vs {category_2}")
print(f"T-Statistic: {t_stat:.4f}")
print(f"P-Value: {p_value:.4f}")

if p_value < 0.05:
    print("The difference in review scores between the two categories is statistically significant.")
else:
    print("The difference in review scores between the two categories is not statistically significant.")


*7. Seller State*

In [ ]:
# Step 1: Calculate average review score per seller state
avg_review_score_per_state = data.groupby('seller_state')['review_score'].mean().sort_values(ascending=False)

# Step 2: Select top 5 and bottom 5 states
top_5_states = avg_review_score_per_state.head(5)
bottom_5_states = avg_review_score_per_state.tail(5)

# Step 3: Combine the top 5 and bottom 5 into one DataFrame
top_and_bottom_5 = pd.concat([top_5_states, bottom_5_states])

# Step 4: Plot the bar plot
plt.figure(figsize=(12, 6))
sns.barplot(
    x=top_and_bottom_5.index,
    y=top_and_bottom_5.values,
    palette='coolwarm'
)
plt.title('Top 5 and Bottom 5 Seller States by Average Review Score', fontsize=16)
plt.xlabel('Seller State', fontsize=12)
plt.ylabel('Average Review Score', fontsize=12)
plt.xticks(rotation=90, ha='right')  # Rotate state names for better readability
plt.show()


In [ ]:
# Step 1: Choose two seller states to compare
category_1 = 'MG'
category_2 = 'RJ'

# Step 2: Filter data for the two selected seller states
category_1_scores = data[data['seller_state'] == category_1]['review_score']
category_2_scores = data[data['seller_state'] == category_2]['review_score']

# Step 3: Perform t-test
t_stat, p_value = ttest_ind(category_1_scores, category_2_scores, equal_var=False)  # Use Welch's t-test

# Step 4: Display results
print(f"T-Test Results for {category_1} vs {category_2}")
print(f"T-Statistic: {t_stat:.4f}")
print(f"P-Value: {p_value:.4f}")

if p_value < 0.05:
    print("The difference in review scores between the two seller states is statistically significant.")
else:
    print("The difference in review scores between the two seller states is not statistically significant.")


*8. Delivery Time*

In [ ]:
# Calculate the average delivery time for each review score
avg_delivery_time = data.groupby('review_score')['delivery_time'].mean().reset_index()

# Bar plot
plt.figure(figsize=(12, 6))
sns.barplot(data=avg_delivery_time, x='review_score', y='delivery_time', palette='Set2')
plt.title('Average Delivery Time by Review Score', fontsize=16)
plt.xlabel('Review Score', fontsize=12)
plt.ylabel('Average Delivery Time (Days)', fontsize=12)
plt.show()


In [ ]:
# Step 1: Define the threshold for "on-time" vs "late"
# We'll use the median delivery time as the cutoff
median_delivery_time = data['delivery_time'].median()

# Step 2: Split the data into two groups based on the delivery time threshold
on_time_reviews = data[data['delivery_time'] <= median_delivery_time]['review_score']
late_reviews = data[data['delivery_time'] > median_delivery_time]['review_score']

# Step 3: Perform T-Test
t_stat, p_value = ttest_ind(on_time_reviews, late_reviews, equal_var=False)

# Step 4: Print Results
print("T-Test: On-time vs Late Deliveries")
print(f"T-Statistic: {t_stat:.4f}")
print(f"P-Value: {p_value:.4f}")

# Interpretation
if p_value < 0.05:
    print("The difference in review scores between on-time and late deliveries is statistically significant.")
else:
    print("The difference in review scores between on-time and late deliveries is not statistically significant.")


*9. Delivery Delay*

In [ ]:
# Define late deliveries (e.g., deliveries with delay > 0)
data['late_delivery'] = data['delivery_delay'] > 0

# Calculate the proportion of late deliveries by review score
late_delivery_proportion = data.groupby('review_score')['late_delivery'].mean().reset_index()

# Bar plot
plt.figure(figsize=(12, 6))
sns.barplot(data=late_delivery_proportion, x='review_score', y='late_delivery', palette='Set2')
plt.title('Proportion of Late Deliveries by Review Score', fontsize=16)
plt.xlabel('Review Score', fontsize=12)
plt.ylabel('Proportion of Late Deliveries', fontsize=12)
plt.show()


In [ ]:
# Step 1: Define On-time and Late Deliveries
# Late deliveries are where the delivery delay is greater than 0 (i.e., late deliveries)
data['late_delivery'] = data['delivery_delay'] > 0

# Step 2: Split the data into on-time and late deliveries
on_time_reviews = data[data['late_delivery'] == False]['review_score']
late_reviews = data[data['late_delivery'] == True]['review_score']

# Step 3: Perform T-Test
t_stat, p_value = ttest_ind(on_time_reviews, late_reviews, equal_var=False)

# Step 4: Print Results
print("T-Test: On-time vs Late Deliveries")
print(f"T-Statistic: {t_stat:.4f}")
print(f"P-Value: {p_value:.4f}")

# Interpretation
if p_value < 0.05:
    print("The difference in review scores between on-time and late deliveries is statistically significant.")
else:
    print("The difference in review scores between on-time and late deliveries is not statistically significant.")


*10. Freight Price Ratio*

In [ ]:
# Calculate the average freight price ratio for each review score
avg_freight_price_ratio = data.groupby('review_score')['freight_price_ratio'].mean().reset_index()

# Bar plot
plt.figure(figsize=(12, 6))
sns.barplot(data=avg_freight_price_ratio, x='review_score', y='freight_price_ratio', palette='Set2')
plt.title('Average Freight Price Ratio by Review Score', fontsize=16)
plt.xlabel('Review Score', fontsize=12)
plt.ylabel('Average Freight Price Ratio (Freight Value / Price)', fontsize=12)
plt.show()

In [ ]:
# Step 1: Define high and low freight price ratio (e.g., greater than 1 is high)
threshold = 1
data['high_freight_ratio'] = data['freight_price_ratio'] > threshold

# Step 2: Split data into high and low freight price ratio
high_freight_reviews = data[data['high_freight_ratio'] == True]['review_score']
low_freight_reviews = data[data['high_freight_ratio'] == False]['review_score']

# Step 3: Perform T-Test
t_stat, p_value = ttest_ind(high_freight_reviews, low_freight_reviews, equal_var=False)

# Step 4: Print results
print("T-Test: High vs Low Freight Price Ratio")
print(f"T-Statistic: {t_stat:.4f}")
print(f"P-Value: {p_value:.4f}")

# Interpretation
if p_value < 0.05:
    print("The difference in review scores between high and low freight price ratio is statistically significant.")
else:
    print("The difference in review scores between high and low freight price ratio is not statistically significant.")


*11. Product Name Length*

In [ ]:
# Create bins for product name length to group similar ranges
data['product_name_length_bins'] = pd.cut(data['product_name_lenght'], bins=10)

# Box plot of review scores across product name length ranges
plt.figure(figsize=(14, 8))
sns.boxplot(data=data, x='product_name_length_bins', y='review_score', palette='Set2')
plt.title('Review Score Distribution Across Product Name Length Ranges', fontsize=16)
plt.xlabel('Product Name Length Range', fontsize=12)
plt.ylabel('Review Score', fontsize=12)
plt.xticks(rotation=45, ha='right')  # Rotate length bins for readability
plt.show()


In [ ]:
# Step 1: Define a threshold for product name length
threshold = 40

# Step 2: Split the data into two groups based on the threshold
short_names = data[data['product_name_lenght'] <= threshold]['review_score']
long_names = data[data['product_name_lenght'] > threshold]['review_score']

# Step 3: Perform t-test
t_stat, p_value = ttest_ind(short_names, long_names, equal_var=False)  # Use Welch's t-test

# Step 4: Display results
print("T-Test Results for Product Name Length (Short vs. Long):")
print(f"T-Statistic: {t_stat:.4f}")
print(f"P-Value: {p_value:.4f}")

if p_value < 0.05:
    print("The difference in review scores between short and long product names is statistically significant.")
else:
    print("The difference in review scores between short and long product names is not statistically significant.")


*12. Product Weight*

In [ ]:
# Create bins for product weight
data['product_weight_bins'] = pd.cut(data['product_weight_g'], bins=10, labels=False)  # 10 bins, labeled numerically

# Calculate average review scores for each product weight range
avg_review_score_by_weight = data.groupby('product_weight_bins')['review_score'].mean().reset_index()

# Bar plot of average review score across product weight ranges
plt.figure(figsize=(14, 8))
sns.barplot(data=avg_review_score_by_weight, x='product_weight_bins', y='review_score', palette='coolwarm')
plt.title('Average Review Score Across Product Weight Ranges', fontsize=16)
plt.xlabel('Product Weight Range (g)', fontsize=12)
plt.ylabel('Average Review Score', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.show()


In [ ]:
# Step 1: Define a weight threshold
threshold = 1000   # 10000 grams (10kg) as the threshold for light vs. heavy products

# Step 2: Split the data into two groups based on the threshold
light_products = data[data['product_weight_g'] <= threshold]['review_score']
heavy_products = data[data['product_weight_g'] > threshold]['review_score']

# Step 3: Perform t-test
t_stat, p_value = ttest_ind(light_products, heavy_products, equal_var=False)  # Use Welch's t-test

# Step 4: Display results
print("T-Test Results for Product Weight (Light vs. Heavy):")
print(f"T-Statistic: {t_stat:.4f}")
print(f"P-Value: {p_value:.4f}")

if p_value < 0.05:
    print("The difference in review scores between light and heavy products is statistically significant.")
else:
    print("The difference in review scores between light and heavy products is not statistically significant.")


*13. Review to Delivery Time*

In [ ]:
# Calculate the average review score for each time range
avg_review_score_by_delivery_time = data.groupby('review_to_delivery_time')['review_score'].mean().reset_index()

# Line plot
plt.figure(figsize=(14, 8))
sns.lineplot(data=avg_review_score_by_delivery_time, x='review_to_delivery_time', y='review_score', marker='o', color='green')
plt.title('Average Review Score Across Review to Delivery Time Ranges', fontsize=16)
plt.xlabel('Review to Delivery Time Range (days)', fontsize=12)
plt.ylabel('Average Review Score', fontsize=12)
plt.xticks(rotation=45, ha='right')  # Rotate bins for readability
plt.show()


In [ ]:
# Step 1: Define the Groups (On-time vs Late reviews)
# On-time reviews: review_to_delivery_time <= 0
# Late reviews: review_to_delivery_time > 0
data['late_review'] = data['review_to_delivery_time'] > 0

# Step 2: Split the data into on-time and late reviews
on_time_reviews = data[data['late_review'] == False]['review_score']
late_reviews = data[data['late_review'] == True]['review_score']

# Step 3: Perform T-Test
t_stat, p_value = ttest_ind(on_time_reviews, late_reviews, equal_var=False)

# Step 4: Print Results
print("T-Test: On-time vs Late Reviews")
print(f"T-Statistic: {t_stat:.4f}")
print(f"P-Value: {p_value:.4f}")

# Interpretation
if p_value < 0.05:
    print("The difference in review scores between on-time and late reviews is statistically significant.")
else:
    print("The difference in review scores between on-time and late reviews is not statistically significant.")


Through the above analysis, the following features have been proven to be significant:
1. Customer State
2. Payment Type
3. Payment Installments
4. Freight Value
5. Product Category
6. Seller State
7. Delivery Time
8. Delivery Delay
9. Freight Price Ratio
10. Product Name Length
11. Product Weight
12. Review to Delivery Time


Apart from these, some statistically insignificant features like:
13. Price


will also be included. While these features may not show strong individual relationships with the target variable, machine learning models, particularly non-linear ones like Random Forests and Gradient Boosting, can uncover complex, non-linear interactions that statistical tests might miss. Some features, despite being statistically insignificant, may still hold business relevance or improve model interpretability. Lastly, including all potentially relevant features helps the model generalize better and remain robust to future data changes.

The remaining features were excluded from the analysis because they either showed negligible correlation with the target variable or added redundancy due to high multicollinearity with other included features. Additionally, some features were irrelevant to the problem context or lacked sufficient variance to contribute meaningfully to the predictive model. By excluding these features, we aim to simplify the model, reduce noise, and improve computational efficiency without compromising predictive performance.






# Modelling

In [ ]:
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.ensemble import GradientBoostingClassifier as GBDT
from xgboost import XGBClassifier as XGB
from sklearn.linear_model import LogisticRegression as LogR
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
from sklearn.model_selection  import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.metrics import ConfusionMatrixDisplay as CM
from sklearn.svm import SVC

In [ ]:
# List of features to keep
features_to_keep = [
    'customer_state',                      # Customer State
    'payment_type',                        # Payment Type
    'payment_installments',                # Payment Installments
    'price',                               # Price
    'freight_value',                       # Freight Value
    'product_category_name_english',       # Product Category
    'seller_state',                        # Seller State
    'delivery_time',                       # Delivery Time
    'delivery_delay',                      # Delivery Delay
    'freight_price_ratio',                 # Freight Price Ratio
    'product_name_lenght',                 # Product Name Length
    'product_weight_g',                    # Product Weight
    'review_to_delivery_time',             # Review from Delivery Time
    'review_score'                         # Review Score
]

# Select the relevant columns
data_prepared = data[features_to_keep]


In [ ]:
# Print info of the filtered data
print(data_prepared.info())

In [ ]:
# One-hot encoding for payment_type, customer_state, product_category_name_english and seller_state columns
encoded_data = pd.get_dummies(data_prepared, columns=['payment_type', 'customer_state','product_category_name_english','seller_state'], drop_first=True)

# Verifying the result of one-hot encoding
print(f"Columns after one-hot encoding:\n{encoded_data.columns.tolist()}")

In [ ]:
# Classifying reviews into 2 categories
encoded_data['review_classified'] = encoded_data['review_score'].apply(lambda x: 0 if x <= 3 else 1)

# Verifying Result
print(encoded_data.head())

# Dropping original review score column
encoded_data = encoded_data.drop(columns=['review_score'])

In [ ]:
# Check class balance
review_score_counts = encoded_data['review_classified'].value_counts()
review_score_proportion = review_score_counts / review_score_counts.sum()
print(review_score_proportion)

In [ ]:
# Separate features and target
features = encoded_data.drop(columns=['review_classified'])
target = encoded_data['review_classified']

# Scale the features
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

# Recreate features DataFrame with scaled data
features = pd.DataFrame(scaled_features, columns=features.columns)

# Split data into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(features, target, test_size=0.2, random_state=1984)

# Check shapes
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)


In [ ]:
# Initialize and train the models
DTC_algo = DTC()
DTC_model = DTC_algo.fit(X_train, Y_train)

svm_algo = SVC()
SVM_modedl = svm_algo.fit(X_train, Y_train)

LogR_algo = LogR()
LogR_model = LogR_algo.fit(X_train, Y_train)

RF_algo = RF()
RF_model = RF_algo.fit(X_train, Y_train)

GBDT_algo = GBDT()
GBDT_model = GBDT_algo.fit(X_train, Y_train)

XGB_algo = XGB()
XGB_model = XGB_algo.fit(X_train, Y_train)

models = [DTC_model,SVM_modedl,LogR_model, RF_model, GBDT_model, XGB_model]
names = ['Decison Tree','SVM','Logistic Regression', 'Random Forest', 'GBDT', 'XGBDT']

for i in range(len(models)):
    print(f"Model: {names[i]}")

    # Predict based on the training data
    predict = models[i].predict(X_train)

    # Calculate accuracy, precision, recall, and F1-score
    precision, recall, f1_score, _ = precision_recall_fscore_support(Y_train, predict, average='macro')
    accuracy = accuracy_score(Y_train, predict)
    print(f"Macro Accuracy: {accuracy}")
    print(f"Macro Precision: {precision}")
    print(f"Macro Recall: {recall}")
    print(f"Macro F1-score: {f1_score}")
    print("\n")

# Evaluate the models

for i in range(len(models)):
    print(f"Model: {names[i]}")

    # Predict based on the test data
    predict = models[i].predict(X_test)

    # Calculate accuracy, precision, recall, and F1-score
    precision, recall, f1_score, _ = precision_recall_fscore_support(Y_test, predict, average='macro')
    accuracy = accuracy_score(Y_test, predict)
    print(f"Macro Accuracy: {accuracy}")
    print(f"Macro Precision: {precision}")
    print(f"Macro Recall: {recall}")
    print(f"Macro F1-score: {f1_score}")
    print("\n")

    # Confusion matrix
    cm = confusion_matrix(Y_test, predict)
    print(f"Confusion Matrix:\n{cm}")

    # Plot the confusion matrix
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Class 0", "Class 1"], yticklabels=["Class 0", "Class 1"])
    plt.title(f"Confusion Matrix - {names[i]}")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

In [ ]:
# Class imbalance
review_score_counts = encoded_data['review_classified'].value_counts()
review_score_proportion = review_score_counts / review_score_counts.sum()
print(review_score_proportion)

In [ ]:
# Oversampling for class 0
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=1234,sampling_strategy=0.5)
X_train, Y_train = smote.fit_resample(X_train, Y_train)

print(X_train.shape)
print(Y_train.shape)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Create a hyperparameter search function for re-usability
def random_search(algo, hyperparameters, X_train, Y_train):
  # do the search using 5 folds/chunks
  clf = RandomizedSearchCV(algo, hyperparameters, cv=5, random_state=2015,
                          scoring='precision_macro', n_iter=20, refit=True)
  # pass the data to fit/train
  clf.fit(X_train, Y_train)

  return clf.best_params_


# GBDT
GBDT_tuned_parameters = {
    'n_estimators': randint(50, 250), # Draw from a uniform distribution between 50 and 250
    'learning_rate': uniform(loc=0.01, scale=4.99),  # Draw from a uniform distribution between 0.01 and 5
    'criterion': ['friedman_mse', 'squared_error'],
    'max_depth': randint(2, 7)  # Draw from a uniform distribution between 2 and 7
}

GBDT_best_params = random_search(GBDT_algo, GBDT_tuned_parameters, X_train, Y_train)

print("GBDT Best Parameters:", GBDT_best_params)

# Train the models
GBDT_algo = GBDT(**GBDT_best_params)
GBDT_model = GBDT_algo.fit(X_train, Y_train)

print('Model: GBDT Train：')

# predict based on training data
predict = GBDT_model.predict(X_train)

# Calculate precision, recall, and F1-score
precision, recall, f1_score, _ = precision_recall_fscore_support(Y_train, predict, average='macro')
accuracy = accuracy_score(Y_train, predict)
print(f"Macro Accuracy: {accuracy}")
print(f"Macro Precision: {precision}")
print(f"Macro Recall: {recall}")
print(f"Macro F1-score: {f1_score}")
print("\n")

print('Model: GBDT Test：')

# predict based on test data
predict = GBDT_model.predict(X_test)

# Calculate precision, recall, and F1-score
precision, recall, f1_score, _ = precision_recall_fscore_support(Y_test, predict, average='macro')
accuracy = accuracy_score(Y_test, predict)
print(f"Macro Accuracy: {accuracy}")
print(f"Macro Precision: {precision}")
print(f"Macro Recall: {recall}")
print(f"Macro F1-score: {f1_score}")
print("\n")

# Confusion matrix
cm = confusion_matrix(Y_test, predict)
print(f"Confusion Matrix:\n{cm}")

# Plot the confusion matrix
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Class 0", "Class 1"], yticklabels=["Class 0", "Class 1"])
plt.title(f"Confusion Matrix - GBDT")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()